# Data Wrangling Project - Open Street Map
__Brent Nixon, Dec. 2017__

## Table of Contents

* <a href='#Introduction'>Introduction</a>
* <a href='#Extracting Data from OSM'>Extracting Data from OSM</a>
* <a href='#Overview of Data and Checking for Problems'>Overview of Data and Checking for Problems</a>
* <a href='#Data Processing and Cleaning'>Data Processing and Cleaning</a>
* <a href='#Loading Data into SQL'>Loading Data into SQL</a>
* <a href='#Exploration of Data with SQL'>Exploration of Data with SQL</a>
* <a href='#Worthy Options for Further Cleaning'>Worthy Options for Further Cleaning</a>

Reference:
http://sebastianraschka.com/Articles/2014_ipython_internal_links.html#top

## Introduction <a id='Introduction'></a>

>In this project, I will examine a section of the Open Street Map (OSM). To do this, I will extract the data from OSM, process the data programatically, look for issues in the data (such as cleanliness, uniformity, and validity), load the data into a local SQL database, and then explore the data using SQL to get an overview of the area.

>I chose to examine OSM data from the southeast part of Charlotte, North Carolina, which is where I grew up. 

>Navigate to this link to examine the bounding box describing the area I worked with: http://www.openstreetmap.org/relation/177415#map=11/35.2033/-80.8401

>Charlotte is an interesting city because it has a wide variety of urbanism. There is a dense urban core with high-rises, semi-urban mixed residential and commercial areas, sub-urban residential areas, and low-density sprawl. For a mapping project, there would certainly be a variety of features and idiosyncrasies to sort out. 

>This cell sets the path for the Jupyter notebook so it knows where to find the supporting files.

In [1]:
# set working directory
import os
PATH= "/Users/brentan/Documents/DAND/Projects/Data_Wrangling_Project/Wrangle_OSM_BNixon"

os.chdir(PATH)
print(os.listdir('.'))

['ways_tags.csv', 'Charlotte_AOI.xml', 'nodes_tags.csv', '.DS_Store', 'nodes.csv', 'charlotte_osm.db', 'Wrangle_OSM_BNixon.ipynb', '__pycache__', 'data_wrangling_schema.sql', 'Charlotte_sample.xml', 'ways.csv', '.ipynb_checkpoints', 'SQL_code_for_charlotte_osm_db.sql', 'ways_nodes.csv', 'clt_osm_db_Keys.txt', 'schema.py']


## Extracting Data from OSM <a id='Extracting Data from OSM'></a>
>The first step in this process is to obtain the data from OSM, we need to make an HTTP request using the Overpass API. The function below, "get_XML_data" takes an OSM url showing the location of interest, and writes the streaming requests object line by line to a file. 

*The code below is adapted from the Udacity case study helper.*

In [2]:
## code for getting XML data from OSM using the Overpass API
import requests
def get_XML_data(URL, FILENAME): 
    
    # make the request but have it stream instead of reading all data into memory at once
    r = requests.get(URL, stream=True)
    
    try: 
        # print the URL to debug in event that error gets thrown 
        print("Request URL:",r.url)

        # Throw an error for bad status codes
        r.raise_for_status()
        
        # use iter_lines to parse each line one by one
        events = r.iter_lines() 
        
        # write each line, line by line, into a writable file
        with open (FILENAME, 'w') as f:
            for line in events:
                f.write(line.decode('utf-8'))
                f.write("\n") # this is necessary to maintain the formatting in the file
                
        # success messages        
        print("File write was success!")
        
    except Exception as e:
        print (e)
        
    finally: 
        r.close()

>The code below will use the "get_XML_data" function to pull the data from OSM for the area of interest in Charlotte. For our final work, we want to work with at least a 50mb file, but to start out, I will explore the data and test my code using a 5-10mb file. The code below will create a sample file that takes elements from throughout the project file to give an accurate representation of issues that will be found in the data.

In [4]:
## Code for sample acquisition
SAMPLE_URL = "http://overpass-api.de/api/map?bbox=-80.8676,35.1993,-80.8099,35.2218"
SAMPLE_FILENAME = "Charlotte_sample.xml"
# get_XML_data(SAMPLE_URL, SAMPLE_FILENAME)

# Code for 50mb size dataset
URL = "http://overpass-api.de/api/map?bbox=-80.8934,35.1157,-80.7368,35.2310"
FILENAME = "Charlotte_AOI.xml"
# get_XML_data(URL, FILENAME)

Request URL: http://overpass-api.de/api/map?bbox=-80.8934,35.1157,-80.7368,35.2310
File write was success!


## Overview of Data and Checking for Problems <a id='Overview of Data and Checking for Problems'></a>
>Now that I have a sample XML file with data from Charlotte, I want to get a rough overview of the data using the way and node tags. This will help me have a working understanding of what my code is checking and what kind of data cleaning I might need to do. 

>The code below will iterate through the XML file finding tags. If a tag name is new, it will add that name to a dictionary. Otherwise, it will add one to the count for that tag type. 

In [5]:
## code block to explore element type and counts 
import xml.etree.cElementTree as ET  
import pprint

# function to list what element names are in the XML file and give their counts
def count_tags(filename):
    things = {}
    for _, val in ET.iterparse(filename):
        if val.tag in things:
            things[val.tag] +=1
        else:
            things[val.tag] = 1
    return things

# execute function on XML file
pprint.pprint(count_tags("Charlotte_AOI.xml"))

{'bounds': 1,
 'member': 10600,
 'meta': 1,
 'nd': 306987,
 'node': 270362,
 'note': 1,
 'osm': 1,
 'relation': 171,
 'tag': 77094,
 'way': 31370}


#### Overview of Data Structure

>From the above code, we can see that there are a variety of different tags. The primary tags, however, are nd, node, tag, member, way, and relation. 

>From the OSM documentation, we see that the basic structure of OSM data is comprised of nodes, ways, and relations. Nodes are points on the map, ways are an ordered list of connected nodes (describing things like roads, rivers, etc...), and relations are composed of related members (nodes and ways). 

>Tag elements are sub-elements of each way, node, and relation, whose attributes provide further information that describe the way, node, or relation. Nd elements are sub-elements of ways, which reference the node elements that compose the way. 

>The last few elements, bounds,meta, note, and osm are all part of the header information of the XML document. The bounds tag contains coordinates showing the map area that the data is from. 

>*Reference:*

>*http://wiki.openstreetmap.org/wiki/Map_Features*

 #### Checking for Compound Values and Problem Characters
>In some of the tags, there are attributes that have values containing two pieces of information in one text string, e.g. "addr:street" instead of "address" and "street." When we bring that information into a SQL databases, it would be preferable to have those two pieces of information separated. 

>In the code below, I will use regular expressions to identify elements whose "k" attributes have a structure with colons as specified above, as well as attributes which do not and attributes which have problematic characters like #, ?, etc... This will help me isolate and correct those issues. 

>*The code below is adapted from the Udacity case study helper.*

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
## code block to identify and count attribute keys with colon structure or problematic characters

import xml.etree.cElementTree as ET
import pprint
import re

# define regular expressions for compiling
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# function to give a count of each of the four tag categories
def count_key_type(element, keys): 
    if element.tag == "tag":
        if lower.search(element.get('k')):
            keys["lower"] += 1
        elif lower_colon.search(element.get('k')):
            keys["lower_colon"] += 1
        elif problemchars.search(element.get('k')):
            keys["problemchars"] += 1
        else: 
            keys["other"] += 1
    pass
    return keys

# function to iteratively process lines in XML file and yield dictionary with count of key types
def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = count_key_type(element, keys)
    return keys

# execute process_map on XML file
keys = process_map('Charlotte_AOI.xml')
pprint.pprint(keys)

{'lower': 51567, 'lower_colon': 23439, 'other': 2088, 'problemchars': 0}


#### Running the code above shows there are:
>* 51567 "k" attributes that are lower case letters and do not have any colons or problematic characters, there are 
>* 23439 "k" attributes with the colon structure
>* 0 "k" attributes with problematic characters
>* 2088 "k" attributes that don't fit into either of those three categories.

#### Examining Different Tag Types
>Now I'm curious what the values in each of those categories look like. I will modify the above code to add each type of attribute value to a list. Then I can print a sample of those values to get a better idea of what they are. 

In [7]:
## code block to identify different types of node and way tag key-attribute values, as well as 
## identifying colon-structured values and problem characters

import xml.etree.cElementTree as ET
import pprint
import re

# define regular expressions for compiling
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# function to identify values with colons and problem characters
def get_key_type(element, key_vals): 
    if element.tag == "tag":
        if lower.search(element.get('k')):
            key_vals["lower"].append(element.get('k'))
        elif lower_colon.search(element.get('k')):
            key_vals["lower_colon"].append(element.get('k'))
        elif problemchars.search(element.get('k')):
            key_vals["problemchars"].append(element.get('k'))
        else: 
            key_vals["other"].append(element.get('k'))
    pass
    return key_vals

# function to iterate through each line of XML file and yield dictionary of lists with values of each type
def process_map(filename):
    key_vals = {"lower": [], "lower_colon": [], "problemchars": [], "other": []}
    for _, element in ET.iterparse(filename):
        key_vals = get_key_type(element, key_vals)
    return key_vals

# execute process_map function on XML file
key_vals = process_map('Charlotte_AOI.xml')

# print out different types of values
print("sample of 'lower' values:")
pprint.pprint(key_vals["lower"][0::2100])
print("\n", "sample of 'lower_colon' values:")
pprint.pprint(key_vals["lower_colon"][0::1000])
print("\n", "sample of 'other' values:")
pprint.pprint(key_vals["other"][0::120])

sample of 'lower' values:
['ele',
 'highway',
 'highway',
 'lanes',
 'highway',
 'highway',
 'source',
 'bicycle',
 'building',
 'width',
 'lanes',
 'building',
 'building',
 'building',
 'height',
 'building',
 'highway',
 'height',
 'building',
 'building',
 'building',
 'access',
 'building',
 'service',
 'surface']

 sample of 'lower_colon' values:
['gnis:id',
 'addr:state',
 'tiger:county',
 'tiger:name_base',
 'tiger:cfcc',
 'tiger:reviewed',
 'tiger:county',
 'tiger:reviewed',
 'tiger:cfcc',
 'tiger:zip_left',
 'tiger:reviewed',
 'tiger:name_base',
 'tiger:name_base',
 'tiger:reviewed',
 'tiger:zip_left',
 'tiger:zip_left',
 'tiger:county',
 'tiger:reviewed',
 'hgv:national_network',
 'tiger:zip_right',
 'tiger:county',
 'tiger:reviewed',
 'tiger:name_type',
 'tiger:name_base']

 sample of 'other' values:
['gnis:Class',
 'gnis:Class',
 'gnis:Class',
 'gnis:Class',
 'tiger:name_base_1',
 'name_1',
 'HFCS',
 'tiger:name_base_2',
 'NCOS:RIV_BASIN',
 'NCOS:RIV_BASIN',
 'NCOS:SPOPL',

>From the above experiment, here are a few take-aways:

>* The tags in the "lower" category do not appear to have any problems. 


>* In the "other" category, there are some attributes that were flagged because they weren't lower case. From the sample, it appears that these represent different acronyms, which are reasonable to be uppercase. 


>* In the "lower_colon" category, there are some values that I want to mess with, and some that I don't! Quite a few values are from US Census Tiger dataset, and plenty from the USGS Geographic Names Information System (GNIS). These values contain useful information, and could be used to create rich units of address information. 

>*References:*
>* *http://wiki.openstreetmap.org/wiki/TIGER*
>* *http://wiki.openstreetmap.org/wiki/USGS_GNIS*

 #### Getting an Overview of Street Information
>Before cleaning, I first need to see what street types are present in the data. The code below will iterate through the sample, looking for any tags with attributes containing street information. This will help me see what potential issues there are in terms of inconsistent street naming or other data quality issues.

In [ ]:
## code block to collect all the values with street information, and to isolate all values with problematic 
## street information
import xml.etree.cElementTree as ET
from collections import defaultdict, OrderedDict
import re
import pprint

# define regular expression to identify street types
osmfile = "Charlotte_AOI.xml"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# dictionary of desired mapping to use when updating street types
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Ext": "Extension",
            "Ext.": "Extension",
            "Ct": "Court",
            "Ct.": "Court",
            "Pl": "Place",
            "Pl.": "Place",
            "Sq": "Square",
            "Sq.": "Square",
            "Ln": "Lane",
            "Ln.": "Lane",
            "Rd": "Road",
            "Rd.": "Road",
            "Trl": "Trail",
            "Trl.": "Trail",
            "Pkwy": "Parkway",
            "Pkwy.": "Parkway",
            "Cmns": "Commons",
            "Cmns.":"Commons"
            }

# function to identify a 'k' attribute with street information
def is_street_name(elem):     
    return (elem.attrib['k'] == "addr:street")

################
## block to collect any values with street information

# function to identify a record with any street information and add it to a dictionary
def get_any_street_type(all_streets, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        all_streets[street_type].add(street_name)

# function to iterate through XML file, collecting a set of all unique values with street information 
def get_all_streets(osm_file):
    osm_file = open(osmfile, "r")
    all_streets = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)): 
        if elem.tag == "node" or elem.tag == "way" or elem.tag == "relation":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    get_any_street_type(all_streets, tag.attrib['v'])
    osm_file.close()
    return all_streets

##########################
## block to isolate problematic street information

# function to identify a record with unexpected street information and add it to a dictionary
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
# function to iterate through XML file, collecting a set of unique values with unexpected street information 
def audit(osm_file):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)): 
        if elem.tag == "node" or elem.tag == "way" or elem.tag == "relation":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

# function to take a problematic street type and update it to the correct type using the mapping dictionary
def update_name(name, mapping):
    street_bits = name.split(" ")
    if street_bits[-1] in mapping:
        street_bits[-1] = mapping[street_bits[-1]]
    return " ".join(street_bits)

############
# code block executing the above functions

# # execute and print results of get_all_streets on the XML file, shows all values with street information
# pprint.pprint(dict(get_all_streets(osmfile)))

# # execute and print results of audit on the XML file, collects and prints problematic street information
# problematic_street_types = audit(osmfile)
# pprint.pprint(dict(audit(osmfile)))

# # execute the update_name function and print the mapping of old street name to new, updated street name
# for st_type, ways in problematic_street_types.items(): 
#     for name in ways:
#         better_name = update_name(name, mapping)
#         print (name, "=>", better_name) 

In [10]:
pprint.pprint(dict(get_all_streets(osmfile)))

{'4': {'Abbey Pl #4'},
 '4b': {'Sardis Road North Suite 4b'},
 'Ardsley': {'Ardsley'},
 'Ave': {'Winthrop Ave'},
 'Ave.': {'2001 Selwyn Ave.'},
 'Avenue': {'Baldwin Avenue',
            'Central Avenue',
            'Charlottetowne Avenue',
            'Commonwealth Avenue',
            'Craig Avenue',
            'Draper Avenue',
            'East Kingston Avenue',
            'East Park Avenue',
            'Hillside Avenue',
            'Kenwood Avenue',
            'Louise Avenue',
            'Magnolia Avenue',
            'Metropolitan Avenue',
            'North Old Woodward Avenue',
            'Pecan Avenue',
            'Radcliffe Avenue',
            'Roswell Avenue',
            'Scott Avenue',
            'Seigle Avenue',
            'Selwyn Avenue',
            'Skyland Avenue',
            'Toomey Avenue',
            'Union Pacific Avenue',
            'Vail Avenue',
            'West Kingston Avenue',
            'West Park Avenue',
            'West Tremont Avenue'},


>Overall, the streets and their abbreviations look correctly spelled and formulated. Other than a few abbreviations that I'd like to see full words for (i.e. Blvd. to Boulevard), there are really no issues.

>Now I want to isolate values with problematic street types to see what kind of issues there are

In [11]:
# execute and print results of audit on the XML file
problematic_street_types = audit(osmfile)
pprint.pprint(dict(audit(osmfile)))

{'4': {'Abbey Pl #4'},
 '4b': {'Sardis Road North Suite 4b'},
 'Ardsley': {'Ardsley'},
 'Ave': {'Winthrop Ave'},
 'Ave.': {'2001 Selwyn Ave.'},
 'Blvd': {'Blythe Blvd', 'South Blvd', 'East Blvd'},
 'Dr': {'Montford Dr'},
 'Ext': {'West 4th Street Ext'},
 'North': {'Sardis Road North'},
 'Pl': {'Harrisonwoods Pl'},
 'Plaza': {'The Plaza'},
 'Rd': {'Old Nations Ford Rd', 'Monroe Rd', 'Albemarle Rd'},
 'St': {'W Hill St'},
 'Way': {'Iverson Way'}}


>There are only twelve instances of abbreviation, and the code below will use the "*update_name*" function to change them to a non-abbreviated form. I did find a street abbreviation, "Ext," that wasn't in my street type mapping, so I added it and its pair (Extension) to the dictionary.

In [12]:
# execute the update_name function and print the mapping of old street name to new, updated street name
for st_type, ways in problematic_street_types.items(): 
    for name in ways:
        better_name = update_name(name, mapping)
        print (name, "=>", better_name) 

Winthrop Ave => Winthrop Avenue
Ardsley => Ardsley
2001 Selwyn Ave. => 2001 Selwyn Avenue
Blythe Blvd => Blythe Boulevard
South Blvd => South Boulevard
East Blvd => East Boulevard
Iverson Way => Iverson Way
Sardis Road North => Sardis Road North
Old Nations Ford Rd => Old Nations Ford Road
Monroe Rd => Monroe Road
Albemarle Rd => Albemarle Road
Sardis Road North Suite 4b => Sardis Road North Suite 4b
Montford Dr => Montford Drive
W Hill St => W Hill Street
Harrisonwoods Pl => Harrisonwoods Place
Abbey Pl #4 => Abbey Pl #4
The Plaza => The Plaza
West 4th Street Ext => West 4th Street Extension


## Data Processing and Cleaning <a id='Data Processing and Cleaning'></a>

>The code below defines the functions that process the XML data and perform all the cleaning processes described above

In [1]:
## code block to process XML data, unbundle colon-structured values, split the element name and attributes into 
## dictionaries, validate that the resulting dictionaries are in the correct structure and data type for their 
## destination SQL tables, and finally, write the dictionaries to CSV files

import csv
import codecs
from collections import defaultdict, OrderedDict
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema # note: refers to the schema.py file attached in this directory  

OSM_PATH = "Charlotte_AOI.xml"

# CSV files to write to
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

# define regular expression to compile, for identifying street type, colon-structured values, and problem characters
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# schema to validate dictionaries against
SCHEMA = schema.schema

# right order of columns for each dict/CSV file
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp'] 
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type'] 
WAY_NODES_FIELDS = ['id', 'node_id', 'position'] 
       
# list of expected street types, used when identifying problematic street types
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# dictionary of desired mapping to use when updating street types
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Ext": "Extension",
            "Ext.": "Extension",
            "Ct": "Court",
            "Ct.": "Court",
            "Pl": "Place",
            "Pl.": "Place",
            "Sq": "Square",
            "Sq.": "Square",
            "Ln": "Lane",
            "Ln.": "Lane",
            "Rd": "Road",
            "Rd.": "Road",
            "Trl": "Trail",
            "Trl.": "Trail",
            "Pkwy": "Parkway",
            "Pkwy.": "Parkway",
            "Cmns": "Commons",
            "Cmns.":"Commons"
            }

# function to process each 'tag' sub-element, extracting the attributes, cleaning problematic street types
# and unbundling any colon-structured values
def shape_tag(element, tags, mapping, node_tags_fields=NODE_TAGS_FIELDS, problem_chars=PROBLEMCHARS, lower_colon=LOWER_COLON, default_tag_type='regular'): 
    for elem in element.iter(tag="tag"): 
        if elem.attrib['k'] == "addr:street":
            m = street_type_re.search(elem.attrib['v'])
            if m:
                street_type = m.group()
                if street_type in expected:
                    continue
                else:
                    old_street_name = elem.attrib['v']
                    street_bits = old_street_name.split(" ")
                    if street_bits[-1] not in mapping:
                        continue
                    else:
                        street_bits[-1] = mapping[street_bits[-1]]
                        elem.attrib['v'] = " ".join(street_bits)
    for elem in element.iter(tag="tag"): 
        tag_dict= {} 
        for field in node_tags_fields: 
            tag_dict[field] = "" 
        tag_dict["id"] = element.get("id") 
        for a, b in elem.items(): 
            if a == "k": 
                if problem_chars.search(b): 
                    continue 
                elif lower_colon.search(b): 
                    splist = b.split(":", maxsplit=1) 
                    tag_dict["key"] = splist[1] 
                    tag_dict["type"] = splist[0] 
                else: 
                    tag_dict["key"] = b 
                    tag_dict["type"] = default_tag_type 
            elif a == "v": 
                    tag_dict["value"] = b 
        tags.append(tag_dict)

# function to process and clean elements, extracting node, way, and nd attributes, and processing tags in the same
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS, way_node_fields=WAY_NODES_FIELDS,\
                  node_tags_fields=NODE_TAGS_FIELDS, problem_chars=PROBLEMCHARS, lower_colon=LOWER_COLON, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {} 
    way_attribs = {}
    way_nodes = []
    tags = []  
   
    if element.tag == "node":   
        for field in node_attr_fields: 
            node_attribs[field] = "" 
        for a, b in element.items():
            if a in node_attr_fields: 
                node_attribs[a] = b 
        shape_tag(element, tags, mapping, node_tags_fields=NODE_TAGS_FIELDS, problem_chars=PROBLEMCHARS, lower_colon=LOWER_COLON, default_tag_type='regular') #run the shape tag function on any tag sub-element
                                   
    if element.tag == "way": 
        for field in way_attr_fields: 
            way_attribs[field] = "" 
        for a, b in element.items(): 
            if a in way_attr_fields: 
                way_attribs[a]= b 
        shape_tag(element, tags, mapping, node_tags_fields=NODE_TAGS_FIELDS, problem_chars=PROBLEMCHARS, lower_colon=LOWER_COLON, default_tag_type='regular') #run the shape tag function on any tag sub-element
        
        for nd in element.iter(tag="nd"): 
            nd_dict= {} 
            nd_dict["id"] = element.get("id") 
            nd_dict["node_id"] = nd.get("ref") 
            nd_dict["position"] = list(element.iter()).index(nd) 
            way_nodes.append(nd_dict)
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_tags': tags, 'way_nodes': way_nodes}
 

# ================================================== #
#               Helper Functions                     #
# ================================================== #

# function to iteratively parse the XML file and yield each element
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

# function to validate if element matches the schema
def validate_element(element, validator, schema=SCHEMA):
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string)) 

# extends csv.DictWriter to handle Unicode input
class UnicodeDictWriter(csv.DictWriter, object):
    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #

# function to iteratively process each element in XML file and write to CSVs
def process_map(file_in, validate):

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
            codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
            codecs.open(WAYS_PATH, 'w') as ways_file, \
            codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
            codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:
    
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader() 
        node_tags_writer.writeheader() 
        ways_writer.writeheader() 
        way_nodes_writer.writeheader() 
        way_tags_writer.writeheader() 

        validator = cerberus.Validator() 
        
        
        count = 0
        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags']) 


if __name__ == '__main__':
    process_map(OSM_PATH, validate=True) 

## Loading Data into SQL<a id='Loading Data into SQL'></a>
>Now that I have pulled the parent and child information for the way and node elements into CSV files, I now want to take the data in those CSV files and load them into a SQL database for further analysis. To do this, I need to create the SQL database, define the schemas for and create the tables that will populate the database, and then load the CSVs into their respective SQL tables.

>I used sqlite3 on my local machine to create the database and tables, and to load the data. After defining the schemas and loading the CSVs one-by-by, several times, I used a link the Udacity Project Details to pull a file made by Stephen Welch that has all the schemas pre-defined.\* I modified that sql file to create the tables, load the CSVs, delete records with header information, and create a few views, using this code:

```SQL
.read SQL_code_for_charlotte_osm_db.sql
```

>Using this code made the process a lot faster and more repeatable! Thanks Alvin Alexander.*


>references: 
>* https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f
>* https://alvinalexander.com/android/sqlite-script-read-execute-how

## Exploration of Data with SQL <a id='Exploration of Data with SQL'></a>
<a id='SQL Table Keys'></a>
### Table Reference Guide
>I've found it very helpful to have a fingertip reference of the structure of my tables nad their primary/foreign keys. Here are the tables for **'charlotte_osm.db'**:

nodes:                    
    - id (primary key)
    - lat 
    - lon
    - user
    - uid 
    - version
    - changeset
    - timestamp 

nodes_tags:
    - id (foreign key, nodes)
    - key
    - value
    - type 
    
ways:
    - id (primary key)
    - user
    - uid
    - version
    - changeset
    - timestamp
    
ways_tags:
    - id (foreign key, ways)
    - key
    - value
    - type

ways_nodes:
    - id (foreign key, ways)
    - node_id (foreign key, nodes (id))
    - position

### Questions to Answer:
>* Get an overview of the types of elements and their numbers
    * Spot check number of: nodes, ways, unique users
    
    
>* How many unique users are there?


>* Who are the top users and how much do they contribute compared to the other users?


>* What are the main types of nodes, ways, and relations, i.e. what type are they?


>* What kind of street information is there now?
    * Can you make good composite addresses using more fields?
    
>* Are there any further issues with street names?
    * Can I find other street names in other fields?
    
>* What fields contain postal codes? 
    * Do the postal codes look good?
    
>* Is there more TIGER, GNIS, or normal OSM feature data?


#### How many nodes, ways, and way nodes are there?

>How many ways?

```SQL
SELECT COUNT(*) WaysCount FROM ways;

WaysCount
----------
31370
```

>How many nodes are there?

```SQL
SELECT COUNT(*) NodesCount FROM nodes;

NodesCount
----------
270362
```

>Each way is composed of various nodes, with the sub-element tag of 'Nd.' How of these sub-elements are there for *ways_nodes*?

```SQL
SELECT COUNT(*) as NdSubEl_Count FROM ways_nodes;

NdSubEl_Count
-------------
306987
```

>How many tag elements are there for *nodes* and *ways*?

```SQL
SELECT COUNT(*) as TagsCount FROM (
        SELECT *
        FROM nodes_tags
            UNION ALL 
        SELECT *
        FROM ways_tags) as w_n_tags;

TagsCount
----------------
76340
```

>There are a lot of nodes, and a lot of descriptive tags and Nd elements. Nodes and their associated Nd sub-elements are by far the most numerous elements. There are not as many ways and relations, which makes sense, as there are bound to be more individual points on the map than there are ways connecting them and relations between them. 

>How many unique people have edited the map area in question? 

```SQL
SELECT COUNT(DISTINCT(waysnnodes.uid))
FROM (SELECT nodes.uid, nodes.user from nodes UNION ALL SELECT ways.uid, ways.user FROM ways) waysnnodes;
```

Ans: 289

>Let's take a look at a sample of those users. 

```SQL 
SELECT DISTINCT waysnnodes.user, waysnnodes.uid
FROM (SELECT nodes.uid, nodes.user from nodes UNION ALL SELECT ways.uid, ways.user FROM ways) waysnnodes
LIMIT 15
;
```

```SQL
user                  uid
--------------------  -------------------------
davidearl             3582
Edward                364
suodrak               630540
fbthies               939079
nyuriks               339581
woodpeck_repair       145231
pratikyadav           2905914
woodpeck_fixbot       147510
bdiscoe               402624
Munchabunch           414318
rickmastfan67         252811
Omnific               1408522
KristenK              1494110
SEVEN                 127989
Dami_Tn               2012449
```

>Let's see which of those users are the top contributors, based on the number of edits. 

>First, I'll create a view for users and the count of their contributions.

```SQL
CREATE VIEW UserContributionTotals AS
SELECT COUNT(waysnnodes.uid) as UserTotals, waysnnodes.user
FROM (
    SELECT nodes.uid, nodes.user 
    FROM nodes 
    UNION ALL 
    SELECT ways.uid, ways.user 
    FROM ways) waysnnodes
GROUP BY waysnnodes.uid
ORDER BY COUNT(waysnnodes.uid) DESC
;
```
>Then query the view for the top 20 contributors.

```SQL
SELECT UserTotals, user
FROM (SELECT * FROM UserContributionTotals LIMIT 20)
;
```
>These are the top 20 contributors, by number of edits. As you can see, the distribution of top contributors is very skewed, with the top three contributing an order of magnitude more than any others.

```SQL
UserTotals            user
--------------------  -------------------------
165243                Omnific
47550                 woodpeck_fixbot
21933                 jumbanho
7222                  HeyYoJimbo
5349                  bdiscoe
5025                  Yoshinion
4466                  Becker_MN_Import_Acc
3220                  WashuOtaku
2954                  Fortepc
2694                  Bancheee
2495                  tobin
1815                  maxerickson
1803                  Brian F
1787                  bot-mode
1748                  woodpeck_repair
1422                  andrewpmk
1334                  Jruze
1283                  rickmastfan67
1223                  emacsen_dwg
1211                  botdidier2020
```

> How many total user contributions are there for this area?

```SQL
SELECT SUM(UserTotals)
FROM (SELECT * FROM UserContributionTotals)
;
```
> Ans: 301732

> How many contributions did the top three users make?

```SQL
SELECT SUM(UserTotals)
FROM (SELECT * FROM UserContributionTotals LIMIT 3)
;
```

> Ans: 234726 contributions, or ~78% of the contributions, which suggests a Pareto relationship between users and mapping contributions.

> What are the main types of nodes and ways, i.e. what do those elements represent on the map?

> Before examining this information further, I will combine the *nodes_tags* and *ways_tags* tables, which have the same <a href='#SQL Table Keys'>structure</a>, in order to get a more complete picture of what these data sets hold. 

```SQL
CREATE VIEW nodeANDway_tags AS
        SELECT *
        FROM nodes_tags
            UNION ALL 
        SELECT *
        FROM ways_tags
;
```

> What categories of tag 'type' are there?

```SQL
SELECT type FROM nodeANDway_tags GROUP BY type;

type
-------------------------
addr
area
brand
building
census
communication
contact
cost
cycleway
demolished
destination
diet
gnis
healthcare
hgv
internet_access
is_in
lanes
maxspeed
mtb
name
note
payment
railway
regular
roof
service
source
tiger
toilets
tower
turn
wheelchair
```

> Originally, the type field was meant to unbundle colon-bundled address information like "addr:street," "addr:house," etc... It appears that many more pieces of colon-bundled information got swept up by the *"shape_tag"* helper function of the *"shape_element"* function.

> Let's see what 'keys' were captured for the addr type.

```SQL
SELECT type, key FROM nodeANDway_tags WHERE type == 'addr' GROUP BY key;

type        key
----------  --------------------
addr        city
addr        country
addr        door
addr        floor
addr        housename
addr        housenumber
addr        housenumber_1
addr        place
addr        postcode
addr        state
addr        street
addr        street_1
addr        unit
```

> There are 12 different keys tied to the 'addr' type. The values associated with each of these keys will be helpful in compiling complete address information for a node. 

> The code below will show the records pertaining to one particular unit of address information.

```SQL
SELECT * FROM nodeANDway_tags WHERE type == addr ORDER BY id LIMIT 5;

id          key                   value                 type
----------  --------------------  --------------------  ----------
38619400    city                  Charlotte             addr
38619400    housenumber           2365                  addr
38619400    postcode              28266                 addr
38619400    state                 NC                    addr
38619400    street                Carmel Road           addr
```

> #### What values are there for:
> * state?
> * postcode?
> * house number?
> * city?
> * country?

```SQL
SELECT COUNT(*), key, value, type
FROM nodeANDway_tags
WHERE key LIKE '%state'
GROUP BY value
;

COUNT(*)              key                        value       type
--------------------  -------------------------  ----------  ----------
222                   state                      NC          addr
```

> I wouldn't expect there to be other states than NC, but it was worth checking. 

> What range of values are there for postal codes? Are there any obvious errors?

```SQL
SELECT COUNT(*), key, value, type
FROM nodeANDway_tags
WHERE key LIKE '%postcode'
GROUP BY value
ORDER BY value 
;

COUNT(*)    key         value       type
----------  ----------  ----------  ----------
5           postcode    28023       addr
42          postcode    28202       addr
62          postcode    28203       addr
17          postcode    28204       addr
31          postcode    28205       addr
12          postcode    28207       addr
7           postcode    28208       addr
55          postcode    28209       addr
22          postcode    28210       addr
14          postcode    28211       addr
1           postcode    28211-4411  addr
4           postcode    28212       addr
8           postcode    28217       addr
5           postcode    28226       addr
2           postcode    28227       addr
1           postcode    28244       addr
1           postcode    28266       addr
10          postcode    28270       addr
1           postcode    28273       addr
1           postcode    28274       addr
1           postcode    48009       addr
1           postcode    NC          addr
1           postcode    NC 28202    addr
1           postcode    NC28209     addr
```

>There are a few postal codes that stand out: 28023, 48009, and, obviously, 28211-4411, NC, NC 28202, and NC28209. The last four are clearly errors or non-uniform data. Regarding the first two, 28023 is probably a typing error, as it is a zip code for a <a href='https://www.google.com/maps/place/China+Grove,+NC+28023/@35.5746279,-80.7341066,11z/data=!3m1!4b1!4m5!3m4!1s0x8853f7bf66dda875:0xb46ea0adc083b48f!8m2!3d35.5805091!4d-80.5993854'>town</a> that is an hour away, and 28203 is a zip code close to downtown Charlotte. 48009 is definitely an error, since it is a postal code for <a href='https://www.google.com/maps/place/Birmingham,+MI+48009/@42.5456942,-83.2342796,14z/data=!3m1!4b1!4m5!3m4!1s0x8824c7a8176abb85:0x8ed1a286ce141f4a!8m2!3d42.5410436!4d-83.2131206'>Birmingham, Michigan</a>.

> How about the values for housenumber?

```SQL
SELECT COUNT(*), key, value, type
FROM nodeANDway_tags
WHERE key LIKE '%housenumber'
GROUP BY value
ORDER BY value 
;
```

> Nothing remarkable in the results of that query.

> Cities, in the city of Charlotte?

```SQL
SELECT COUNT(*), key, value, type
FROM nodeANDway_tags
WHERE key LIKE '%city'
GROUP BY value
ORDER BY value 
;

COUNT(*)    key         value                 type
----------  ----------  --------------------  ----------
2           capacity    4                     regular
1           capacity    5                     regular
1           capacity    55                    regular
1           city        Charlote              addr
306         city        Charlotte             addr
1           city        Charlotte, NC         addr
```

> Just a spelling error and some extra information. Only one record with each error!


> Last one, any unusual countries?

```SQL
SELECT COUNT(*), key, value, type
FROM nodeANDway_tags
WHERE key LIKE '%country'
GROUP BY value
ORDER BY value 
;

COUNT(*)    key         value                      type
----------  ----------  -------------------------  ----------
6           country     US                         addr
2           country     USA                        addr
```

> All US or USA. It would probably be worth making those entries uniform.

> *Reference:*

> * *SQL 'LIKE '%example' parameter, Udacity SQL Sample Project, https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md*

## Worthy Options for Further Cleaning <a id='Worthy Options for Further Cleaning'></a>
> There were a few data cleanliness issues that I did not address, and which could be good opportunities for further work. For example, I focused on cleaning address information of the OSM feature type, but there are also many records from the Tiger dataset. For example, see this 'highway' record:

```XML
<way id="16668607" version="7" timestamp="2014-08-14T13:49:20Z" changeset="24746007" uid="105454" user="hopfavogl">
    <nd ref="172274468"/>
    <nd ref="3016461626"/>
    <nd ref="172274470"/>
    <nd ref="2401902303"/>
    <nd ref="172274476"/>
    <tag k="highway" v="residential"/>
    <tag k="name" v="Piedmont Street"/>
    <tag k="tiger:cfcc" v="A41"/>
    <tag k="tiger:county" v="Mecklenburg, NC"/>
    <tag k="tiger:name_base" v="Piedmont"/>
    <tag k="tiger:name_type" v="St"/>
    <tag k="tiger:reviewed" v="no"/>
    <tag k="tiger:zip_left" v="28204"/>
    <tag k="tiger:zip_right" v="28204"/>
  </way>
```

> In fact, if I were to focus on more data cleaning and/or compiling detailed addresses for features, it would be nice to know which source of address data has more features, the OSM, GNIS, or Tiger data types. 

```SQL
SELECT type, COUNT(DISTINCT(id)) as Num_Features
FROM nodeANDway_tags
WHERE type = "gnis" OR type = "tiger" OR type = "addr"
GROUP BY type
ORDER BY COUNT(*) DESC
;

type        Num_Features
----------  ---------------
tiger       3105
addr        397
gnis        201
```

> As it turns out, the tiger data has address information for the most features, followed the OSM (addr) type, and then the GNIS type. For the sake of efficiency, it would definitely be worth focusing on the Tiger records. Thus, to more thoroughly clean street information, it would be a very good idea to expand the "*is_street_name*" function to include these records. This would not be a very difficult process.


> Another issue I noticed is in node tags with the key '*is_in*'. They all have the value '*Mecklenburg,North Carolina,N.C.,NC,USA*,' which clearly needs to be cleaned up. See the example element below.

>To fix this issue, one could simply alter four functions: '*is_street_name*,' '*audit_street_type*, '*audit*', and '*update_name*'. Instead of looking for and updating a street type, you would modify the functions to look for 'is_in' attributes and change their values to a clean version of the duplicate-ridden value. 

```XML
  <node id="153902091" lat="35.2136142" lon="-80.8257289" version="4" timestamp="2017-09-24T08:41:42Z" changeset="52322768" uid="364" user="Edward">
    <tag k="ele" v="209"/>
    <tag k="gnis:Class" v="Populated Place"/>
    <tag k="gnis:County" v="Mecklenburg"/>
    <tag k="gnis:County_num" v="119"/>
    <tag k="gnis:ST_alpha" v="NC"/>
    <tag k="gnis:ST_num" v="37"/>
    <tag k="gnis:id" v="1001262"/>
    <tag k="import_uuid" v="bb7269ee-502a-5391-8056-e3ce0e66489c"/>
    <tag k="is_in" v="Mecklenburg,North Carolina,N.C.,NC,USA"/>
    <tag k="name" v="Elizabeth"/>
    <tag k="place" v="hamlet"/>
    <tag k="wikidata" v="Q5362242"/>
  </node>
```

```SQL
SELECT COUNT(*), key, value, type FROM nodeANDway_tags WHERE key == 'is_in';

COUNT(*)    key         value                                     type
----------  ----------  ----------------------------------------  ----------
85          is_in       Mecklenburg,North Carolina,N.C.,NC,USA    regular
```

>Since there are 85 records with this issue, it would definitely be worth doing.